## Using a LSTM to try to generate funk songs.
First, we'll do a bunch of importing

In [16]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import pandas as pd
import random
import sys
import io
import string, re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import itertools

Cleaning the text...

In [17]:
lyrics_df = pd.read_csv('dataset/Dataset_funk_carioca.csv')
lyrics_df.head(10)

,Unnamed: 0,fonte,titulo1,titulo2,artista,letra
0,0,https://www.letras.mus.br/mc-doni/te-amo-sem-c...,Te Amo Sem Compromisso (To Nem Aí),Te Amo Sem Compromisso (To Nem Aí),MC Doni,"\nNão adianta tu se declarar\nRomance, compro..."
1,1,https://www.letras.mus.br/mc-kevin-o-cris/evol...,Evoluiu (part. Sodré),Evoluiu (part. Sodré),MC Kevin o Chris,"\nEvoluiu! Ritmo agressivo, 150 fluiu\nLevand..."
2,2,https://www.letras.mus.br/mc-davi/love-66-part...,"Love 66 (part. MC Rita, MC Hariel e Gaab)","Love 66 (part. MC Rita, MC Hariel e Gaab)",MC Davi,\n[MC Davi]\nVocê me tira do sério\nEu fazia ...
3,3,https://www.letras.mus.br/ludmilla/invocada-pa...,Invocada (part. Léo Santana),Invocada (part. Léo Santana),Ludmilla,"\nEu te amava, não posso negar\nTambém sofria..."
4,4,https://www.letras.mus.br/mc-magal/positividad...,Positividade (part. MC Bola),Positividade (part. MC Bola),MC Magal,\nTá torcendo contra o meu progresso\nTá preo...
5,5,https://www.letras.mus.br/mc-rodson/mas-pode-a...,"Mas Pode Até Falar de Mim, Tô Nem Aí","Mas Pode Até Falar de Mim, Tô Nem Aí",MC Rodson,\nMas só que a mina é muito linda\nE logo ass...
6,6,https://www.letras.mus.br/mc-cabelinho/era-uma...,Poesia Acústica #6 - Era Uma Vez,Poesia Acústica #6 - Era Uma Vez,MC Cabelinho,"\n[MC Cabelinho]\nOh, fé\n(Poesia)\nPassa nad..."
7,7,https://www.letras.mus.br/mc-th/deixa-part-xam...,"Deixa (part. Xamã, WC no Beat e Pep Starling)","Deixa (part. Xamã, WC no Beat e Pep Starling)",MC TH,"\n[MC TH]\nAí, se não for pedir muito\nPega u..."
8,8,https://www.letras.mus.br/mc-kevin-o-cris/hit-...,Hit Contagiante (part. Felipe Original),Hit Contagiante (part. Felipe Original),MC Kevin o Chris,"\nSe liga aí, malvada, no que agora eu vou ma..."
9,9,https://www.letras.mus.br/ludmilla/a-boba-fui-...,A Boba Fui Eu (part. Jão),A Boba Fui Eu (part. Jão),Ludmilla,\nSe era isso que você queria\nUma noite apen...


In [18]:
def remove_punc(lyrics):
    return "".join([c for c in lyrics if c not in string.punctuation])

tokenizer = RegexpTokenizer(r'\w+')

lyrics_df['letra'] = lyrics_df['letra'].apply(lambda x: remove_punc(x))
lyrics_df['letra'] = lyrics_df['letra'].apply(lambda x: tokenizer.tokenize(x))
lyrics_df['letra'].head(10)

0    [Não, adianta, tu, se, declarar, Romance, comp...
1    [Evoluiu, Ritmo, agressivo, 150, fluiu, Levand...
2    [MC, Davi, Você, me, tira, do, sério, Eu, fazi...
3    [Eu, te, amava, não, posso, negar, Também, sof...
4    [Tá, torcendo, contra, o, meu, progresso, Tá, ...
5    [Mas, só, que, a, mina, é, muito, linda, E, lo...
6    [MC, Cabelinho, Oh, fé, Poesia, Passa, nada, e...
7    [MC, TH, Aí, se, não, for, pedir, muito, Pega,...
8    [Se, liga, aí, malvada, no, que, agora, eu, vo...
9    [Se, era, isso, que, você, queria, Uma, noite,...
Name: letra, dtype: object

Formating the text to the lstm...

In [19]:
def format_lyrics(lyrics, maxlen, step):
    text = list(itertools.chain.from_iterable(lyrics))
    text = ' '.join(text)
    chars = sorted(list(set(text)))
    print('total chars:', len(chars))
    char_indices = dict((c, i) for i, c in enumerate(chars))
    indices_char = dict((i, c) for i, c in enumerate(chars))

    # cut the text in semi-redundant sequences of maxlen characters
    
    sentences = []
    next_chars = []
    for i in range(0, len(text) - maxlen, step):
        sentences.append(text[i: i + maxlen])
        next_chars.append(text[i + maxlen])
    print('nb sequences:', len(sentences))

    print('Vectorization...')
    x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
    y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
    for i, sentence in enumerate(sentences):
        for t, char in enumerate(sentence):
            x[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1
        
    return chars, char_indices, indices_char, x, y, text


We'll only use the lyrics from tbh&c

In [20]:
maxlen = 50
step = 3
lyrics = lyrics_df['letra'].values
chars, char_indices, indices_char, x, y, text = format_lyrics(lyrics, maxlen, step)

total chars: 90
nb sequences: 419623
Vectorization...


Now, defining the model and using some functions to visualize the process, based on [this tutorial](https://keras.io/examples/lstm_text_generation/)

In [30]:
print('Build model...')
model = Sequential()
model.add(LSTM(64, input_shape=(maxlen, len(chars)), return_sequences=True))
model.add(Dropout(0.))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.))
model.add(LSTM(64))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.

    if(epoch<9):
        return
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

Build model...


In [31]:
model.fit(x, y,
          batch_size=128,
          epochs=10,
          callbacks=[print_callback])

Epoch 1/10
419623/419623 [==============================] - 251s 597us/step - loss: 1.9507
Epoch 2/10
419623/419623 [==============================] - 251s 599us/step - loss: 1.6994
Epoch 3/10
419623/419623 [==============================] - 253s 602us/step - loss: 1.6648
Epoch 4/10
419623/419623 [==============================] - 253s 604us/step - loss: 1.7613
Epoch 5/10
419623/419623 [==============================] - 253s 604us/step - loss: 2.1658
Epoch 6/10
419623/419623 [==============================] - 259s 618us/step - loss: 3.2388
Epoch 7/10
419623/419623 [==============================] - 275s 655us/step - loss: 2.8384
Epoch 8/10
419623/419623 [==============================] - 266s 633us/step - loss: 2.9063
Epoch 9/10
419623/419623 [==============================] - 252s 600us/step - loss: 2.9230
Epoch 10/10
419623/419623 [==============================] - 242s 578us/step - loss: 2.6728
----- Generating text after Epoch: 9
----- diversity: 0.2
----- Generating with seed: "qu